In [ ]:
import deeppavlov
import json
#import numpy as np
import tensorflow as tf

#from itertools import chain
#from pathlib import Path

In [ ]:
from deeppavlov.core.commands.train import build_model_from_config
from deeppavlov.core.data.dataset_reader import DatasetReader
#from deeppavlov.core.data.utils import download_decompress
from deeppavlov.core.common.registry import register

In [ ]:
@register('char_lm_dataset_reader')
class CharLmDatasetReader(DatasetReader):
    """
    Dataset reader which reads 
    """
    def read(self, dir_path: str, mode='self_original'):
        dir_path = Path(dir_path)
        dataset = {}
        for dt in ['train', 'valid', 'test']:
            dataset[dt] = self._parse_data(dir_path / '{}_{}.txt'.format(dt, mode))

        return dataset

    @staticmethod
    def _parse_data(filename):
        examples = []
        print(filename)
        curr_persona = []
        curr_dialog_history = []
        persona_done = False
        with filename.open('r') as fin:
            for line in fin:
                line = ' '.join(line.strip().split(' ')[1:])
                your_persona_pref = 'your persona: '
                if line[:len(your_persona_pref)] == your_persona_pref and persona_done:
                    curr_persona = [line[len(your_persona_pref):]]
                    curr_dialog_history = []
                    persona_done = False
                elif line[:len(your_persona_pref)] == your_persona_pref:
                    curr_persona.append(line[len(your_persona_pref):])
                else:
                    persona_done = True
                    x, y, _, candidates = line.split('\t')
                    candidates = candidates.split('|')
                    example = {
                        'persona': curr_persona,
                        'x': x,
                        'y': y,
                        'dialog_history': curr_dialog_history[:],
                        'candidates': candidates,
                        'y_idx': candidates.index(y)
                    }
                    curr_dialog_history.extend([x, y])
                    examples.append(example)

        return examples

In [1]:
print(len('DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'))

85


In [4]:
a = 'a' / 'b'
print(a)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [7]:
def greeting(name: str) -> str:
    return name

print(greeting(1))

1


In [8]:
a, b = (1, 2)

In [9]:
print(a)

1


In [6]:
print(0 or 3)

3


In [7]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer

tokenizer = RussianTokenizer(ngram_range=[1, 1])
batch = ['abc', 'def']
print(tokenizer(batch))

2018-08-01 15:29:52.751 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/anton/dpenv/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-08-01 15:29:52.805 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


[['abc'], ['def']]


In [9]:
from deeppavlov.models.tokenizers.utils import ngramize

items = ['abcd', 'efgh']
for i in ngramize(items, ngram_range=(2, 2)):
    print(i)

['abcd efgh']


In [17]:
def f(a, b, c='3'):
    """Function test its fields"""
    d = a + b + c
    return d

print(f)
print(f.__call__.__call__.__call__.__call__.__call__.__name__)
print(f.__name__)
print(f.__doc__)
print(f.__defaults__)
print(f.__kwdefaults__)
print(f.__code__)

<function f at 0x7ff83c3356a8>
__call__
f
Function test its fields
('3',)
None
<code object f at 0x7ff83c326e40, file "<ipython-input-17-3e15ce66597b>", line 1>


In [23]:
def f():
    print(f)
f()

<function f at 0x7ff83c335e18>


In [28]:
x = input("Do you need the answer? (y/n): ")
if x.lower() == "y":
    required = True
else:
    required = False
    
def the_answer(self, *args):              
        return 42
    
class EssentialAnswers(type):
    
    def __init__(cls, clsname, superclasses, attributedict):
        if required:
            cls.the_answer = the_answer
                           
    
class Philosopher1(metaclass=EssentialAnswers): 
    pass


print(Philosopher1.the_answer)
class Philosopher2(metaclass=EssentialAnswers): 
    pass
class Philosopher3(metaclass=EssentialAnswers): 
    pass
    
    
plato = Philosopher1()
print(plato.the_answer())
kant = Philosopher2()
# let's see what Kant has to say :-)
print(kant.the_answer())

Do you need the answer? (y/n): y


AttributeError: type object 'EssentialAnswers' has no attribute 'the_answer'

In [29]:
from char_lm_vocab import CharLMVocabulary
vocab = CharLMVocabulary(
    save_path='./vocab.dict',
    load_path='./vocab.dict',
)
text = 'DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'
vocab.fit(text)
vocab.save()


2018-08-03 15:36:34.385 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.400 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.412 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 59: [saving vocabulary to /home/anton/DeepPavlov/download/vocab.dict]


In [31]:
print(len(vocab))
print(vocab[5])
print('z' in vocab)
print('a' in vocab)
print(vocab.is_str_batch([]))
print(vocab.is_str_batch(['c']))
print(vocab(['a', 'b', 'я']))

27
r
False
True
False
True
[3, 14, None]


In [1]:
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as LSTM
import numpy as np

num_units = 100
voc_size = 20
inps = tf.placeholder(tf.int32, shape=[None, None])
lbls = tf.placeholder(tf.int32, shape=[None, None])
prep_inps = tf.one_hot(inps, voc_size)
prep_lbls = tf.one_hot(lbls, voc_size)

out_w = tf.Variable(
    tf.truncated_normal([num_units, voc_size])
)

with tf.device('/gpu:0'):
    lstm = LSTM(
        2,
        num_units,
        input_mode='skip_input',
    )

bs = tf.shape(inps)[1:-1]
state_shape = tf.concat([[2], bs, [num_units]], 0)
with tf.device('/gpu:0'):
    lstm_res, state = lstm(
        prep_inps
    )
    print(lstm_res)
    logits = tf.matmul(tf.reshape(lstm_res, [-1, num_units]), out_w)
    predictions = tf.nn.softmax(logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits,
        labels=tf.reshape(prep_lbls, [-1, voc_size])
    ))
    opt = tf.train.GradientDescentOptimizer(1.)
    train_op = opt.minimize(loss)
    
nu = 10
bs = 32
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        inputs = np.random.randint(0, voc_size, size=(nu, bs))
        labels = np.random.randint(0, voc_size, size=(nu, bs))
        _, preds = sess.run([train_op, preds], feed_dict={inps: inputs, lbls: labels})

    


KeyboardInterrupt: 

In [3]:
import itertools

def create_distribute_map(num_distributed, result_length):
    quotient = result_length / num_distributed
    num_filled = 0
    if num_distributed < result_length:
        num_repeats = list()
        for i in range(1, num_distributed):
            filled_space = quotient * i
            print(filled_space)
            truncated = int(filled_space)
            num_repeats.append(truncated - num_filled)
            num_filled = truncated
        num_repeats.append(result_length - num_filled)
    else:
        num_repeats = [1] * result_length
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(num_repeats)]))
    return map_

print(create_distribute_map(3, 2))

[0, 1]


In [6]:
for idx in range(10, -1, -1):
    print(idx)

10
9
8
7
6
5
4
3
2
1
0


In [17]:
with open('debug.txt', 'r') as f:
    t = f.read()
print(t.split('\n')[0])
print('"%s"' % t)
l = []
print(l[0])


""


IndexError: list index out of range

In [7]:
import itertools
import copy
from useful_functions import all_combs

def create_even_distribution(num_unique_items, num_slots):
    """Returns a list with number of slots taken by each item. If num_slots < num_unique_items not all items
    will be distributed"""
    items_per_slot_float = num_slots / num_unique_items
    total_slots_filled_saved = 0
    if num_unique_items < num_slots:
        item_distribution = list()
        for i in range(1, num_unique_items):
            total_slots_float = items_per_slot_float * i
            total_slots_filled = int(total_slots_float)
            item_distribution.append(total_slots_filled - total_slots_filled_saved)
            total_slots_filled_saved = total_slots_filled
        item_distribution.append(num_slots - total_slots_filled_saved)
    else:
        item_distribution = [1] * num_slots
    return item_distribution


def create_distribute_map(num_unique_items, num_slots):
    item_distribution = create_even_distribution(num_unique_items, num_slots)
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(item_distribution)]))
    return map_


def approx_mem_consumption(config):
    # in MB for sequence_length=100, batch_size=128, num_layers=2, num_units=2000
    # working for all lstm variants
    base_consumption = 4500
    consumption = base_consumption * \
        (config['num_units'] / 2000)**2 * \
        (config['num_layers'] / 2) * \
        (config['batch_size'] / 128) * \
        (config['sequence_length'] / 100)
    return consumption


def num_consequent_repeats(list_, idx):
    current = list_[idx]
    i = 1
    while list_[idx+i] == current:
        i += 1
    return i


def ceil(a):
    if int(a) != a:
        return int(a) + 1
    return int(a)


def split_experiment_config_into_separate_measurement_configs(config):
    seq_lens = make_list(config['sequence_length'])
    batch_sizes = make_list(config['batch_size'])
    num_layers = make_list(config['num_layers'])
    num_units = make_list(config['num_units'])
    combs = all_combs([seq_lens, batch_sizes, num_layers, num_units])
    configs = list()
    for comb in combs:
        conf = copy.deepcopy(config)
        conf['sequence_length'] = comb[0]
        conf['batch_size'] = comb[1]
        conf['num_layers'] = comb[2]
        conf['num_units'] = comb[3]
        configs.append(conf)
    if config['num_repeats'] == 1:
        return configs
    else:
        confs = list()
        for conf in configs:
            confs.extend([conf]*config['num_repeats'])
        return confs


def get_configs_run_in_parallel(configs, counter):
    current_config = configs[counter]
    max_num_in_parallel = current_config['memory_per_experiment'] / approx_mem_consumption(current_config)
    num_repeats = num_consequent_repeats(configs, counter)
    configs_to_process = configs[counter:counter+num_repeats]
    num_launches_required = ceil(num_repeats / max_num_in_parallel)
    distribution = create_even_distribution(num_launches_required, num_repeats)
    list_of_launches = list()
    pointer = 0
    for num_proc in distribution:
        list_of_launches.append(configs_to_process[pointer:pointer+num_proc])
        pointer += num_proc
    return list_of_launches, num_repeats


def make_list(candidate):
    if isinstance(candidate, list):
        l = candidate
    else:
        l = [candidate]
    return l

In [16]:
config = {
  "num_layers": [3, 4],
  "num_units": [1000, 2000],
  "sequence_length": [50, 100],
  "batch_size": [128],
  "save_path": "results/lstm_test",
  "lstm_type": "cudnn_stacked",
  "mode": "train",
  "num_steps": 500,
  "num_proc_in_pool": 5,
  "num_repeats": 10,
  "memory_per_experiment": 7500
}

In [17]:
configs = split_experiment_config_into_separate_measurement_configs(config)
launches = get_configs_run_in_parallel(configs, 0)
for idx, launch in enumerate(launches[0]):
    print(idx)
    for conf in launch:
        print(conf)
    print('\n')

0
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats'

In [18]:
print(approx_mem_consumption(configs[0]))

843.75


In [1]:
for i in range(20):
    print(2**(2*i) - 2**i + 2)

2
4
14
58
242
994
4034
16258
65282
261634
1047554
4192258
16773122
67100674
268419074
1073709058
4294901762
17179738114
68719214594
274877382658


In [2]:
for i in range(20):
    print(i**2 - i + 2)

2
2
4
8
14
22
32
44
58
74
92
112
134
158
184
212
242
274
308
344


In [3]:
import tensorflow as tf
from tensorflow.nn.rnn_cell import LSTMCell as LSTMCell
lstms = [LSTMCell(num_units, dtype=tf.float32) for num_units in [100, 200, 300]]
multilayer_lstm = tf.contrib.rnn.MultiRNNCell(lstms)
print(multilayer_lstm.zero_state(10, tf.float32))

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState/zeros:0' shape=(10, 100) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState/zeros_1:0' shape=(10, 100) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_1/zeros:0' shape=(10, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_1/zeros_1:0' shape=(10, 200) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_2/zeros:0' shape=(10, 300) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_2/zeros_1:0' shape=(10, 300) dtype=float32>))


In [4]:
print(tf.__version__)

1.10.0


In [6]:
a = tf.Variable(0, trainable=False, validate_shape=False)

In [7]:
b = ([1], [2], [3])
for k in zip(*b):
    print(k)

(1, 2, 3)


In [13]:
def deep_zip(objects, depth, passable_types=(list, tuple, dict)):
    if depth != 0 and isinstance(objects[0], passable_types):
        if isinstance(objects[0], (list, tuple)):
            zipped = list()
            for comb in zip(*objects):
                zipped.append(
                    deep_zip(comb, depth-1, passable_types=passable_types)
                )
            if isinstance(objects[0], tuple):
                zipped = tuple(zipped)
            return zipped
        elif isinstance(objects[0], dict):
            zipped = dict()
            for key in objects[0].keys():
                values = [obj[key] for obj in objects]
                zipped[key] = deep_zip(values, depth-1, passable_types=passable_types)
            return zipped
    return tuple(objects)

In [28]:
objects = [
    [(1, 2), ([3, 9], 4), {'a': 4}],
    [(5, 6), ([7, 10], 8), {'a': 5}],
]
depth = 10
print(deep_zip(objects, depth, passable_types=(list, tuple)))

[((1, 5), (2, 6)), ([(3, 7), (9, 10)], (4, 8)), ({'a': 4}, {'a': 5})]


In [22]:
def apply_func_on_depth(obj, func, depth, passable_types=(list, tuple, dict)):
    if depth != 0 and isinstance(obj, passable_types):
        if isinstance(obj, (list, tuple)):
            processed = list()
            for elem in obj:
                processed.append(apply_func_on_depth(elem, func, depth-1, passable_types=passable_types))
            if isinstance(obj, tuple):
                processed = tuple(processed)
            return processed
        elif isinstance(obj, dict):
            processed = dict()
            for key, value in obj.items():
                processed[key] = apply_func_on_depth(value, func, depth-1, passable_types=passable_types)
            return processed
    return func(obj)

In [27]:
obj = [
    [(1, 2), ([3, 9], 4), {'a': 4}],
    [(5, 6), ([7, 10], 8), {'a': 5}],
]

func = lambda x: x**2
depth = 4
print(apply_func_on_depth(obj, func, depth))

[[(1, 4), ([9, 81], 16), {'a': 16}], [(25, 36), ([49, 100], 64), {'a': 25}]]


In [1]:
import tensorflow as tf
a = tf.Variable(0)
ndim_tensor = tf.shape(tf.shape(a))
is_scalar = tf.equal(tf.shape(tf.shape(a))[0], 0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(is_scalar))

True


In [2]:
f = lambda: 1
print(f())

1


In [6]:
a = {'a': 1, 'b': 2, 'c': 3}
def f(*args):
    print(args)

f(*a.items())

(('a', 1), ('b', 2), ('c', 3))


In [3]:
def f(a=1):
    print(a)
    
kwargs = {'a': 2}

f(4, **kwargs)

TypeError: f() got multiple values for argument 'a'

In [7]:
d = {([1, 2], [3, 4]): 1, ([5, 6], [7, 8]): 2}


TypeError: unhashable type: 'list'

In [13]:
print(not 0)

True


In [14]:
a = [1, 2]
print(a['1'])

TypeError: list indices must be integers or slices, not str

In [9]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

# no exception
sv = SimpleVocabulary(
    load_path="file_which_does_not_exists",
    save_path="file_which_does_not_exists"
)
print(sv.load_path.is_file())
sv.load()
print(len(sv))

print()
# exception
sv = SimpleVocabulary(
    load_path="no_such_dir/file_which_does_not_exists",
    save_path="no_such_dir/file_which_does_not_exists"
)
print(sv.load_path.parent.is_dir())
print(sv.load_path.is_file())
sv.load()
print(len(sv))


False
0

True
False
0


In [3]:
import os
try:
    user_paths = os.environ['PYTHONPATH'].split(os.pathsep)
except KeyError:
    user_paths = []
print(user_paths)

[]


In [11]:
from collections import Counter
from itertools import chain
l = [1, 2, 3, 2, 1, 1]
freqs = Counter(chain(l))
print(list(freqs.most_common()))

[(1, 3), (2, 2), (3, 1)]


In [5]:
import re
s0 = 'abc\ndef'
s1 = repr(s0)
print(s1)
s2 = bytes(s, "utf-8").decode("unicode_escape")
print(s2)

'abc\ndef'
abc
def


In [7]:
def f(a: int) -> int:
    print(a)
    return [3]
    
f([1, 2])


[1, 2]


[3]

In [18]:
import numpy as np
vocab = ['a', 'b', 'c']
a = np.random.rand(10, 3)
def f(vec):
    return vocab[np.argmax(vec)]
b = np.apply_along_axis(f, -1, a)
print(''.join(b))
print(b.dtype.name)
print(np.dtype(str).name)
print(np.dtype(np.array(['a', 'b'])))

ccaccbaaba
str32
str


TypeError: data type not understood

In [23]:
import numpy as np
a = np.array([['a', 'b']])
print(a.dtype.name)

str32


In [6]:
from tensorflow.nn.rnn_cell import LSTMStateTuple as LSTMStateTuple
t1 = LSTMStateTuple(h='b', c='a')
t2 = LSTMStateTuple(c='a', h='b')
print(isinstance(t1, tuple))
for e in t1:
    print(e)
for e in zip(t1, t2):
    print(e)

True
a
b
('a', 'a')
('b', 'b')


In [38]:
import tensorflow as tf


def is_scalar_tensor(t):
    return tf.equal(tf.shape(tf.shape(t))[0], 0)


def replace_empty_saved_state(saved_and_zero_state):
    return tf.cond(
        is_scalar_tensor(saved_and_zero_state[0]),
        true_fn=lambda: saved_and_zero_state[1],
        false_fn=lambda: tf.reshape(saved_and_zero_state[0], tf.shape(saved_and_zero_state[1])),
    )

def synchronous_flatten(*nested):
    if not isinstance(nested[0], (tuple, list, dict)):
        return [[n] for n in nested]
    output = [list() for _ in nested]
    if isinstance(nested[0], dict):
        for k in nested[0].keys():
            flattened = synchronous_flatten(*[n[k] for n in nested])
            for o, f in zip(output, flattened):
                o.extend(f)
    else:
        try:
            for inner_nested in zip(*nested):
                flattened = synchronous_flatten(*inner_nested)
                for o, f in zip(output, flattened):
                    o.extend(f)
        except TypeError:
            print('(synchronous_flatten)nested:', nested)
            raise
    return output

def compose_save_list(*pairs, name_scope='save_list'):
    with tf.name_scope(name_scope):
        save_list = list()
        for pair in pairs:
            # print('pair:', pair)
            [variables, new_values] = synchronous_flatten(pair[0], pair[1])
            # print("(useful_functions.compose_save_list)variables:", variables)
            # variables = flatten(pair[0])
            # # print(variables)
            # new_values = flatten(pair[1])
            for variable, value in zip(variables, new_values):
                save_list.append(tf.assign(variable, value, validate_shape=False))
        return save_list

# saved_state = (
#     tf.Variable(0., validate_shape=False, trainable=False, name='state'),
#     tf.Variable(0., validate_shape=False, trainable=False, name='state')
# )
saved_state = tf.Variable(0., validate_shape=False, trainable=False, name='state')

lstm = tf.nn.rnn_cell.LSTMCell(10, state_is_tuple=False)

inp = tf.zeros([100, 10, 13])
zero_state = lstm.zero_state(tf.shape(inp)[1], tf.float32)

# state = [
#     replace_empty_saved_state([s, z]) for s, z in zip(saved_state, zero_state)
# ]
state = replace_empty_saved_state([saved_state, zero_state])
output, new_state = tf.nn.dynamic_rnn(
    lstm, inp, initial_state=state, parallel_iterations=1024, time_major=True
)


opt = tf.train.GradientDescentOptimizer(1.)

save_list = compose_save_list((saved_state, new_state))
with tf.control_dependencies(save_list):
    loss = tf.reduce_sum(output)
    grads_and_vars = opt.compute_gradients(loss, var_list=tf.global_variables())
    grads_and_vars = [(grad, var) for grad, var in grads_and_vars if grad is not None]
    train_op = opt.apply_gradients(grads_and_vars)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        sess.run(train_op)
        print(i)

2018-09-13 15:26:54.204 WARNING in 'tensorflow'['tf_logging'] at line 120: <tensorflow.python.ops.rnn_cell_impl.LSTMCell object at 0x7f8bfc62b940>: Using a concatenated state is slower and will soon be deprecated.  Use state_is_tuple=True.


InvalidArgumentError: var and delta do not have the same shape[10,20] []
	 [[Node: GradientDescent/update_state/ApplyGradientDescent = ApplyGradientDescent[T=DT_FLOAT, use_locking=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](state, GradientDescent/learning_rate, gradients/cond/Reshape/Switch_grad/cond_grad)]]

Caused by op 'GradientDescent/update_state/ApplyGradientDescent', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/anton/dpenv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/anton/dpenv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/anton/dpenv/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/h

In [20]:
import numpy as np
indices_batch = np.array([None, None])
print(indices_batch[0] is None)
a = np.array([1, 2])
for i in np.nditer(indices_batch, flags=['refs_ok']):
    print(i.dtype.name)
for i in np.nditer(a, flags=['refs_ok']):
    print(i.dtype.name)
if any([i is None for i in np.nditer(indices_batch, flags=['refs_ok'])]):
    print('!')
    g = np.vectorize(lambda x: x is None)
    mask = g(indices_batch)
    print(mask)

True
object
object
int64
int64


In [21]:
print(repr('\n'))

'\n'


In [32]:
import numpy as np
a = np.array([[[1, 4], [2, 5], [3, 6]], [[7, 8], [9, 10], [11, 12]]])
b = np.reshape(a, (-1, a.shape[-1]))
correct = sum([y1 == y2 for y1, y2 in zip(a, a)])
for y1, y2 in zip(a, a):
    print(y1, y2, '\n')
print(correct, '\n')
print(len(a))

[[1 4]
 [2 5]
 [3 6]] [[1 4]
 [2 5]
 [3 6]] 

[[ 7  8]
 [ 9 10]
 [11 12]] [[ 7  8]
 [ 9 10]
 [11 12]] 

[[2 2]
 [2 2]
 [2 2]] 

2


In [35]:
import sys
import numpy as np
from deeppavlov.core.common.log import get_logger


def prepare_for_accuracy_computation(y_true, y_predicted):
    log = get_logger(__name__)
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)
    if not isinstance(y_predicted, np.ndarray):
        y_predicted = np.array(y_predicted)
    num_examples = y_true.size
    if y_true.size != y_predicted.size:
        num_examples = min(y_true.size, y_predicted.size)
        log.warning(
            "Number of elements in y_true and in y_predicted are not equal\n"
            "y_true.size = {}, y_predicted.size = {}\n"
            "Using first {} elements of y_true and y_predicted".format(
                y_true.size, y_predicted.size, num_examples,
            )
        )
        y_true, y_predicted = np.reshape(y_true, (-1)), np.reshape(y_predicted, (-1))
        y_true, y_predicted = y_true[:num_examples], y_predicted[:num_examples]
    if y_true.shape != y_predicted.shape:
        log.warning(
            "y_true and y_predicted have different shapes\ny_true.shape = {}, y_predicted.shape = {}\n"
            "Reshaping y_true to y_predicted.shape".format(y_true.shape, y_predicted.shape)
        )
        y_true = np.reshape(y_true, y_predicted.shape)
    return y_true, y_predicted, num_examples


def accuracy(y_true, y_predicted):
    """
    Calculate accuracy in terms of absolute coincidence

    Args:
        y_true: array of true values
        y_predicted: array of predicted values

    Returns:
        portion of absolutely coincidental samples
    """
    
    y_true, y_predicted, num_examples = prepare_for_accuracy_computation(y_true, y_predicted)
    correct = np.sum(y_true == y_predicted)
    return correct / num_examples if num_examples else 0

In [37]:
y_true = np.array([[1, 2, 4]])
y_predicted = []
print(accuracy(y_true, y_predicted))

2018-09-13 14:25:53.122 WARNING in '__main__'['<ipython-input-35-9e32c49161a5>'] at line 22: Number of elements in y_true and in y_predicted are not equal
y_true.size = 3, y_predicted.size = 0
Using first 0 elements of y_true and y_predicted


True
0


In [32]:
print(np.mean(np.array([])))

nan


/home/anton/dpenv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anton/dpenv/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [40]:
a = np.array([1, 2, 3])
l = []
l += list(a)
print(l)

[1, 2, 3]


In [1]:
import numpy as np
a = np.array(['a', 'b'])
for e in np.nditer(a):
    print(type(e))
    print(str(e))

<class 'numpy.ndarray'>
a
<class 'numpy.ndarray'>
b
